In [56]:
# import dependencies
# import pandas as pd
# import numpy as np
import random
import json


In [58]:
# list to hold probabilities for which demographic a new case will belong to
infectProbs = []
# list to hold probability that an infection will result in death for each demographic
deathProbs = []

# populate lists above
for data in stateData[state]:
    infectProbs.append(stateData[state][data]['chance_of_infection'])
    deathProbs.append(stateData[state][data]['chance_of_death'])


In [60]:
# select desired state
state = 'missouri'

# access json file with data to be sonified for avaliable states
stateData = json.load(open('../resources/stateData.json'))

# display dictionary
stateData


{'tennessee': {'white': {'population': 5200531,
   'inverted_ccvi': 45.91,
   'chance_of_infection': 15.44,
   'chance_of_death': 1.47,
   'generated_cases': 0,
   'generated_deaths': 0},
  'black': {'population': 1120221,
   'inverted_ccvi': 28.16,
   'chance_of_infection': 15.65,
   'chance_of_death': 1.68,
   'generated_cases': 0,
   'generated_deaths': 0},
  'native': {'population': 18151,
   'inverted_ccvi': 40.66,
   'chance_of_infection': 9.31,
   'chance_of_death': 1.12,
   'generated_cases': 0,
   'generated_deaths': 0},
  'asian': {'population': 117348,
   'inverted_ccvi': 49.7,
   'chance_of_infection': 9.41,
   'chance_of_death': 0.79,
   'generated_cases': 0,
   'generated_deaths': 0},
  'pacific': {'population': 3735,
   'inverted_ccvi': 37.37,
   'chance_of_infection': 24.52,
   'chance_of_death': 1.03,
   'generated_cases': 0,
   'generated_deaths': 0},
  'hispanic': {'population': 363753,
   'inverted_ccvi': 32.45,
   'chance_of_infection': 25.67,
   'chance_of_death':

In [57]:
# weighted probabilities for #people the current case can spread infection to
# (from left to right, values refer to weights for 0, 1, 2, and 3 new infections)
spreadProbs = [0,0,0,12]
# symptomatic cases that have occured
sympCount = 0
# deaths that have occured
deathCount = 0
# case count threshold for when new r0 shoud be calculated
thresh = 0
# nth threshold
n = 1
# infection rate
r0 = 3
# transposition in semitones
trans = 0
# cases remaining (one case to start it off)
more = [0]
# which case
case = 0
# array of midi notes
midiNotes = [48, 51, 55, 58, 62, 63, 65, 67, 68, 68, 74, 77, 78, 80]
# dictionary with all data
weathering = {
    'time': [],
    'demo': [],
    'note': [],
    'death': []
}


In [59]:
# function to determine how many cases correspond to a particular threshold
def nthThresh(n,t):
    return 100 * n + t

# function to adjust spreadProbs weights and calculate r0 for corresponding threshold
def thisThresh():
    if n == 1:
        spreadProbs[3] -= 1
        spreadProbs[2] += 1
    elif n == 2:
        spreadProbs[3] -= 1
        spreadProbs[1] += 1
    elif n == 13:
        spreadProbs[2] -= 1
        spreadProbs[0] += 1
    elif n == 14:
        spreadProbs[1] -= 1
        spreadProbs[0] += 1
    elif n != 0 and n < 13:
        spreadProbs[3] -= 1
        spreadProbs[0] += 1
    r0 = (spreadProbs[1] + 2*spreadProbs[2] + 3*spreadProbs[3]) / sum(spreadProbs)
    
# function to select rescaled incubation period as a delay in milliseconds.
def incubate(n):
    if n == 1:  
        sub = 3
        return random.randint(0, sub*40)*2000/sub
    elif n == 2:
        sub = random.choice([3,4])
        return random.randint(0, sub*40)*2000/sub
    elif n == 3:
        sub = random.choice([6,4])
        return random.randint(0, sub*40)*2000/sub
    elif n == 4:
        sub = random.choice([6,8])
        return random.randint(0, sub*40)*2000/sub
    elif n == 5:
        sub = random.choice([6,8,5])
        return random.randint(0, sub*40)*2000/sub
    else:
        sub = random.choice([6,8,5,7])
        return random.randint(0, sub*40)*2000/sub
    
# function to select a value based on a probability distribution array
def select(probsArray):
    return random.choices(list(range(len(probsArray))),weights=probsArray)[0]

# function to decide between one and zero based on probability that 1 will be chosen
def decide(prob):
    return random.choices([0,1], weights=[100-prob, prob])[0]

# function to generate midi notes
def midi(t):
    note = random.choice(midiNotes)
    return note + t


In [61]:
while len(more) > 0:
    
    # decide how many people the current case infects and add them to more
    for i in range(select(spreadProbs)):
        more.append(case)
        
    # decide if asymptomatic or not
    if decide(40) == 0:
        
        # add current case to total # of cases
        # (only sympotmatic cases adjust threshold)
        sympCount += 1
        
        # if case count is at or over threshold number
        if sympCount >= nthThresh(n, thresh):
            # calculate new threshold
            thresh = nthThresh(n, thresh)
            # adjust spreadProbs weights and calculate r0 for corresponding threshold
            thisThresh()
            # add one to threshold n value
            n += 1
        
        # select which demographic the current case belongs to
        demo = select(infectProbs)
        weathering['demo'].append(demo)
        
        # select which note
        weathering['note'].append(midi(0))
        
        # select incubation period (delay time)
        delay = incubate(n)
        
        # calculate time from beginning that symptoms occur for current case
        if sympCount == 1:
            time = delay
        else:
            time = delay + weathering['time'][more[0]]
        weathering['time'].append(time)

        # decide/record if case results in death or not
        death = decide(deathProbs[demo])
        weathering['death'].append(death)
        
        case += 1
        
    # remove current case from how many more cases left
    more.pop(0)
    


In [62]:
result = list(zip(weathering['time'], weathering['demo'], weathering['note'], weathering['death']))

result = sorted(result, reverse=False)

len(result)


11321

In [63]:
timing = {}
ms = 0
infections = -1

for time in result:

    if time[0] != ms:
        infections += 1
        timing[infections] = {'demoNotes': [time[1], time[2], time[3]]}
        timing[infections]['delay'] = time[0] - ms
        ms = time[0]

    else:
        for i in range(1,4):
            timing[infections]['demoNotes'].append(time[i])



In [64]:
timing

{0: {'demoNotes': [3, 68, 0], 'delay': 77333.33333333333},
 1: {'demoNotes': [1, 78, 0], 'delay': 6666.6666666666715},
 2: {'demoNotes': [2, 48, 0], 'delay': 80000.0},
 3: {'demoNotes': [3, 74, 0], 'delay': 7333.333333333343},
 4: {'demoNotes': [0, 62, 0], 'delay': 2666.666666666657},
 5: {'demoNotes': [2, 78, 0], 'delay': 2000.0},
 6: {'demoNotes': [2, 78, 0], 'delay': 4000.0},
 7: {'demoNotes': [1, 58, 0, 4, 74, 0], 'delay': 1333.333333333343},
 8: {'demoNotes': [1, 65, 0], 'delay': 2000.0},
 9: {'demoNotes': [5, 74, 0], 'delay': 9333.333333333314},
 10: {'demoNotes': [2, 62, 0], 'delay': 2000.0},
 11: {'demoNotes': [1, 68, 0], 'delay': 333.33333333334303},
 12: {'demoNotes': [5, 58, 0], 'delay': 5000.0},
 13: {'demoNotes': [3, 55, 0], 'delay': 1333.333333333343},
 14: {'demoNotes': [2, 67, 0], 'delay': 666.666666666657},
 15: {'demoNotes': [1, 78, 0, 5, 77, 0], 'delay': 5333.333333333343},
 16: {'demoNotes': [3, 55, 0], 'delay': 1333.333333333314},
 17: {'demoNotes': [3, 67, 0], 'de